In [2]:
## check biased.word.train to confirm simplediff
import simplediff
import re

In [24]:
# filter to have sentence length smaller than 50
pairs_full = []
with open('../cs7650_project_data/biased.full') as f:
    for line in f:
        items = re.sub(' ##','',line).split('\t')
        if len(items[1].split())< 100:
            pairs_full.append((items[1],items[2]))
print('len(pairs_full)',len(pairs_full))

len(pairs_full) 180184


In [25]:
diff_list = [simplediff.string_diff(pair[0],pair[1]) for pair in pairs_full]

In [26]:
diff_list[4][0]

('=',
 ['today', ',', 'on', 'large', 'farms', ',', 'motorcycles', ',', 'dogs', 'or'])

In [27]:
simplediff.string_diff(pairs_full[3][0],pairs_full[3][1])

[('=',
  ['dennis',
   'the',
   'menace',
   'is',
   'an',
   'american',
   'animated',
   'series',
   'produced',
   'by',
   'dic',
   'entertainment',
   ',',
   'based',
   'on',
   'the']),
 ('-', ['classic']),
 ('=', ['comic', 'strip', 'by', 'hank', 'ketcham', '.'])]

In [28]:
simplediff.html_diff(pairs_full[3][0],pairs_full[3][1])

'dennis the menace is an american animated series produced by dic entertainment , based on the <del>classic</del> comic strip by hank ketcham .'

In [34]:
# filter sentence pairs to have len(del) with 2-4 words
diff_pairs = []
idx_diff = [] # keep track of indice in pairs full
for i,diff in enumerate(diff_list):
#     # deletion case
#     if len(diff) == 3 and diff[0][0]=='=' and diff[1][0]=='-' and diff[2][0]=='=':
#         deletion = diff[1][1]
#         if len(deletion) > 1 and len(deletion) < 4:
#             diff_pairs.append((' '.join(deletion),''))
#             idx_diff.append(i)

    # delete and insert
    if len(diff) == 4 and diff[0][0]=='=' and diff[1][0]=='-' and diff[2][0]=='+' and diff[3][0]=='=':
        deletion = diff[1][1]
        insertion = diff[2][1]
        if len(deletion) > 0 and len(deletion) < 6 and len(insertion) > 0 and len(insertion) < 6:
            diff_pairs.append((' '.join(deletion),' '.join(insertion)))
            idx_diff.append(i)

print("len(diff_pairs)",len(diff_pairs))

len(diff_pairs) 44987


In [35]:
# write to file for translation
with open('./bias-unbias.pair','w') as f:
    text = ''
    for a,b in diff_pairs:
        text+=a+'\t'+b+'\n'
    f.write(text)

In [31]:
pairs_full[0]

("during the campaign , controversy erupted over alleged differences between palin ' s positions as a gubernatorial candidate and her position as a vice - presidential candidate .",
 "during the campaign , some pointed out alleged differences between palin ' s positions as a gubernatorial candidate and her position as a vice - presidential candidate .")

In [32]:
# # write to file for Bert Classification corresponding to the bias-unbias pair
# with open('./bias-unbias.pair.sentence','w') as f:
#     text = ''
#     for i in idx_diff:
#         a,b = pairs_full[i]
#         text+=a+'\t'+b+'\n'
#     f.write(text)

import pickle
# save target idx for biased.full and save as pickle for Bert classification data prep
pickle.dump(idx_diff,open("idx_filtered","wb"))